In [240]:
from sympy import *

<font size= 6> Generic case: free surface: </font>

1. `dVxdy = C0*P + C1*dudx + C2*dvdx`
2. `dVydy = D0*P + D1*dudx + D2*dvdx`

see notebook: FreeSurface_v5.ipynb

In [241]:
def eval_dVxdy(dudx,dvdx,P,C0,C1,C2):
    # dVxdy = -dvdx
    dVxdy = (C0*P + C1*dudx + C2*dvdx)
    return dVxdy
def eval_dVydy(dudx,dvdx,P,D0,D1,D2):
    #dVydy = dudx/2 + 3/4*P/eta
    dVydy = (D0*P + D1*dudx + D2*dvdx)
    return dVydy

<font size= 6> Discretisation of the linear momentum balance </font>

In [242]:
ani       = 1
free_surf = 2 # 0: inner points, 1: vertices at surface, 2: centers next to surface

In [243]:
def d_dx2(dudxi,dudeta,dksidx,detadx):
    return dksidx*dudxi + detadx*dudeta
def d_dy2(dvdxi,dvdeta,dksidy,detady):
    return dksidy*dvdxi + detady*dvdeta

In [244]:
dx, dy, dt = symbols('dx, dy, dt')
VxC, VxW, VxE, VxS, VxN = symbols('VxC, VxW, VxE, VxS, VxN')
VyC, VyW, VyE, VyS, VyN = symbols('VyC, VyW, VyE, VyS, VyN')
VySW, VySE, VyNW, VyNE = symbols('VySW, VySE, VyNW, VyNE')
VxSW, VxSE, VxNW, VxNE = symbols('VxSW, VxSE, VxNW, VxNE')
pS, pN, pE, pW = symbols('pS, pN, pE, pW')
D11E, D11W, D12E, D12W, D13E, D13W = symbols('D11E, D11W, D12E, D12W, D13E, D13W')
D21N, D21S, D22N, D22S, D23N, D23S = symbols('D21N, D21S, D22N, D22S, D23N, D23S')
D11N, D11S, D12N, D12S, D13N, D13S = symbols('D11N, D11S, D12N, D12S, D13N, D13S')
D21E, D21W, D22E, D22W, D23E, D23W = symbols('D21E, D21W, D22E, D22W, D23E, D23W')
D31E, D31W, D32E, D32W, D33E, D33W = symbols('D31E, D31W, D32E, D32W, D33E, D33W')
D31N, D31S, D32N, D32S, D33S, D33N = symbols('D31N, D31S, D32N, D32S, D33S, D33N')

In [245]:
dofsV = [VxC, VxW, VxE, VxS, VxN, VxSW, VxSE, VxNW, VxNE,
         VyC, VyW, VyE, VyS, VyN, VySW, VySE, VyNW, VyNE]
dofsP = [ pW, pE, pS, pN ]

In [246]:
# Jacobian terms
aW,bW,cW,dW = symbols('aW,bW,cW,dW')
aE,bE,cE,dE = symbols('aE,bE,cE,dE')
aS,bS,cS,dS = symbols('aS,bS,cS,dS')
aN,bN,cN,dN = symbols('aN,bN,cN,dN')
aC,bC,cC,dC = symbols('aC,bC,cC,dC')
# Free surface for dVxdy
c0C,c1C,c2C = symbols('c0C,c1C,c2C')
c0W,c1W,c2W = symbols('c0W,c1W,c2W')
c0E,c1E,c2E = symbols('c0E,c1E,c2E')
c0S,c1S,c2S = symbols('c0S,c1S,c2S')
c0N,c1N,c2N = symbols('c0N,c1N,c2N')
# Free surface for dVydy
d0C,d1C,d2C = symbols('d0C,d1C,d2C')
d0W,d1W,d2W = symbols('d0W,d1W,d2W')
d0E,d1E,d2E = symbols('d0E,d1E,d2E')
d0S,d1S,d2S = symbols('d0S,d1S,d2S')
d0N,d1N,d2N = symbols('d0N,d1N,d2N')

In [247]:
# Velocity gradients
# 1. `dVxdy = -dVydx`
# 2. `dVydy = 1/2*dVxdx + 3/4*P/eta`
eta_N = symbols('eta_N')
eta_W = symbols('eta_W')
eta_E = symbols('eta_E')
# Velocity gradients on reference grid
if free_surf == 1: # for vertices
    # East
    dVxdxiE  = (VxE - VxC )/dx
    dVydxiE  = (VyE - VyC )/dx
    dVxdetaE = eval_dVxdy(dVxdxiE,dVydxiE,pE,c0E,c1E,c2E) 
    dVydetaE = eval_dVydy(dVxdxiE,dVydxiE,pE,d0E,d1E,d2E)
    # West
    dVxdxiW  = (VxC - VxW )/dx
    dVydxiW  = (VyC - VyW )/dx
    dVxdetaW = eval_dVxdy(dVxdxiW,dVydxiW,pW,c0W,c1W,c2W)
    dVydetaW = eval_dVydy(dVxdxiW,dVydxiW,pW,d0W,d1W,d2W)
    # North
    dVxdxiN  = 0*(VxNE- VxNW)/dx
    dVxdetaN = 0*(VxN - VxC )/dy
    dVydxiN  = 0*(VyNE- VyNW)/dx
    dVydetaN = 0*(VyN - VyC )/dy
    # South
    dVxdxiS  = (VxSE- VxSW)/dx
    dVxdetaS = (VxC - VxS )/dy
    dVydxiS  = (VySE- VySW)/dx
    dVydetaS = (VyC - VyS )/dy
if free_surf == 2: # for centroids
    # East
    dVxdxiE  = (VxE - VxC )/dx
    dVxdetaE = (VxNE- VxSE)/dy
    dVydxiE  = (VyE - VyC )/dx
    dVydetaE = (VyNE- VySE)/dy
    # West
    dVxdxiW  = (VxC - VxW )/dx
    dVxdetaW = (VxNW- VxSW)/dy
    dVydxiW  = (VyC - VyW )/dx
    dVydetaW = (VyNW- VySW)/dy
    # North
    dVxdxiN  = (VxNE- VxNW)/dx
    dVydxiN  = (VyNE- VyNW)/dx
    dVxdetaN = eval_dVxdy(dVxdxiN,dVydxiN,pN,c0N,c1N,c2N)
    dVydetaN = eval_dVydy(dVxdxiN,dVydxiN,pN,d0N,d1N,d2N)
    # South
    dVxdxiS  = (VxSE- VxSW)/dx
    dVxdetaS = (VxC - VxS )/dy
    dVydxiS  = (VySE- VySW)/dx
    dVydetaS = (VyC - VyS )/dy
if free_surf == 0:
    # East
    dVxdxiE  = (VxE - VxC )/dx
    dVxdetaE = (VxNE- VxSE)/dy
    dVydxiE  = (VyE - VyC )/dx
    dVydetaE = (VyNE- VySE)/dy
    # West
    dVxdxiW  = (VxC - VxW )/dx
    dVxdetaW = (VxNW- VxSW)/dy
    dVydxiW  = (VyC - VyW )/dx
    dVydetaW = (VyNW- VySW)/dy
    # North
    dVxdxiN  = (VxNE- VxNW)/dx
    dVxdetaN = (VxN - VxC )/dy
    dVydxiN  = (VyNE- VyNW)/dx
    dVydetaN = (VyN - VyC )/dy
    # South
    dVxdxiS  = (VxSE- VxSW)/dx
    dVxdetaS = (VxC - VxS )/dy
    dVydxiS  = (VySE- VySW)/dx
    dVydetaS = (VyC - VyS )/dy
# Velocity gradients on deformed grid
dVxdxE = d_dx2(dVxdxiE, dVxdetaE, aE,bE)
dVydyE = d_dy2(dVydxiE, dVydetaE, cE,dE)
dVxdxW = d_dx2(dVxdxiW, dVxdetaW, aW,bW)
dVydyW = d_dy2(dVydxiW, dVydetaW, cW,dW)
# ----------
dVxdxN = d_dx2(dVxdxiN, dVxdetaN, aN,bN)
dVydyN = d_dy2(dVydxiN, dVydetaN, cN,dN)
dVxdxS = d_dx2(dVxdxiS, dVxdetaS, aS,bS)
dVydyS = d_dy2(dVydxiS, dVydetaS, cS,dS)
# ----------
dVydxN = d_dx2(dVydxiN, dVydetaN, aN,bN)
dVxdyN = d_dy2(dVxdxiN, dVxdetaN, cN,dN)
dVydxS = d_dx2(dVydxiS, dVydetaS, aS,bS)
dVxdyS = d_dy2(dVxdxiS, dVxdetaS, cS,dS)
# ----------
dVydxE = d_dx2(dVydxiE, dVydetaE, aE,bE)
dVxdyE = d_dy2(dVxdxiE, dVxdetaE, cE,dE)
dVydxW = d_dx2(dVydxiW, dVydetaW, aW,bW)
dVxdyW = d_dy2(dVxdxiW, dVxdetaW, cW,dW)

In [248]:
# Kinematics
divE   = dVxdxE + dVydyE
divW   = dVxdxW + dVydyW
divN   = dVxdxN + dVydyN
divS   = dVxdxS + dVydyS
ExxE   = dVxdxE - Rational(1,3)*divE
ExxW   = dVxdxW - Rational(1,3)*divW
ExxN   = dVxdxN - Rational(1,3)*divN
ExxS   = dVxdxS - Rational(1,3)*divS
EyyE   = dVydyE - Rational(1,3)*divE
EyyW   = dVydyW - Rational(1,3)*divW
EyyN   = dVydyN - Rational(1,3)*divN
EyyS   = dVydyS - Rational(1,3)*divS
ExyN   = Rational(1,2)*(dVxdyN + dVydxN)
ExyS   = Rational(1,2)*(dVxdyS + dVydxS)
ExyE   = Rational(1,2)*(dVxdyE + dVydxE)
ExyW   = Rational(1,2)*(dVxdyW + dVydxW)

In [249]:
# Deviatoric stress
TxxE   =     D11E*ExxE + ani*D12E*EyyE + ani*D13E*ExyE
TxxW   =     D11W*ExxW + ani*D12W*EyyW + ani*D13W*ExyW
TxxN   =     D11N*ExxN + ani*D12N*EyyN + ani*D13N*ExyN
TxxS   =     D11S*ExxS + ani*D12S*EyyS + ani*D13S*ExyS
TyyE   = ani*D21E*ExxE +     D22E*EyyE + ani*D23E*ExyE
TyyW   = ani*D21W*ExxW +     D22W*EyyW + ani*D23W*ExyW
TyyN   = ani*D21N*ExxN +     D22N*EyyN + ani*D23N*ExyN
TyyS   = ani*D21S*ExxS +     D22S*EyyS + ani*D23S*ExyS
TxyE   = ani*D31E*ExxE + ani*D32E*EyyE +     D33E*ExyE
TxyW   = ani*D31W*ExxW + ani*D32W*EyyW +     D33W*ExyW
TxyN   = ani*D31N*ExxN + ani*D32N*EyyN +     D33N*ExyN
TxyS   = ani*D31S*ExxS + ani*D32S*EyyS +     D33S*ExyS

In [250]:
# Stress gradient
dTxxdx = (TxxE - TxxW)/dx
dTxxdy = (TxxN - TxxS)/dy
dTyydx = (TyyE - TyyW)/dx
dTyydy = (TyyN - TyyS)/dy
dTxydx = (TxyE - TxyW)/dx
dTxydy = (TxyN - TxyS)/dy
dpdx   = (pE - pW)/dx
dpdy   = (pN - pS)/dy
# Linear momentum balance
Fx = -( d_dx2(dTxxdx,dTxxdy,aC,bC) + d_dy2(dTxydx,dTxydy,cC,dC) - d_dx2(dpdx,dpdy,aC,bC) )
Fy = -( d_dy2(dTyydx,dTyydy,cC,dC) + d_dx2(dTxydx,dTxydy,aC,bC) - d_dy2(dpdx,dpdy,cC,dC) )

In [251]:
a = ('D11E', 'D11W', 'D12E', 'D12W', 'D13E', 'D13W',
     'D21N', 'D21S', 'D22N', 'D22S', 'D23N', 'D23S',
     'D11N', 'D11S', 'D12N', 'D12S', 'D13N', 'D13S',
     'D21E', 'D21W', 'D22E', 'D22W', 'D23E', 'D23W',
     'D31E', 'D31W', 'D32E', 'D32W', 'D33E', 'D33W',
     'D31N', 'D31S', 'D32N', 'D32S', 'D33S', 'D33N')
b = ('D.D11E', 'D.D11W', 'D.D12E', 'D.D12W', 'D.D13E', 'D.D13W',
     'D.D21N', 'D.D21S', 'D.D22N', 'D.D22S', 'D.D23N', 'D.D23S',
     'D.D11N', 'D.D11S', 'D.D12N', 'D.D12S', 'D.D13N', 'D.D13S',
     'D.D21E', 'D.D21W', 'D.D22E', 'D.D22W', 'D.D23E', 'D.D23W',
     'D.D31E', 'D.D31W', 'D.D32E', 'D.D32W', 'D.D33E', 'D.D33W',
     'D.D31N', 'D.D31S', 'D.D32N', 'D.D32S', 'D.D33S', 'D.D33N')
c = ('aW','bW','cW','dW',
     'aE','bE','cE','dE',
     'aS','bS','cS','dS',
     'aN','bN','cN','dN',
     'aC','bC','cC','dC')
d = ('a.W','b.W','c.W','d.W',
     'a.E','b.E','c.E','d.E',
     'a.S','b.S','c.S','d.S',
     'a.N','b.N','c.N','d.N',
     'a.C','b.C','c.C','d.C')
# for free surface coeffs

e = ('c0C','c1C','c2C',
     'c0W','c1W','c2W',
     'c0E','c1E','c2E',
     'c0S','c1S','c2S',
     'c0N','c1N','c2N')
f = ('fs.C0C','fs.C1C','fs.C2C',
     'fs.C0W','fs.C1W','fs.C2W',
     'fs.C0E','fs.C1E','fs.C2E',
     'fs.C0S','fs.C1S','fs.C2S',
     'fs.C0N','fs.C1N','fs.C2N')
g = ('d0C','d1C','d2C',
     'd0W','d1W','d2W',
     'd0E','d1E','d2E',
     'd0S','d1S','d2S',
     'd0N','d1N','d2N')
h = ('fs.D0C','fs.D1C','fs.D2C',
     'fs.D0W','fs.D1W','fs.D2W',
     'fs.D0E','fs.D1E','fs.D2E',
     'fs.D0S','fs.D1S','fs.D2S',
     'fs.D0N','fs.D1N','fs.D2N')


In [252]:
i=0
for dof in dofsV:
    i+=1
    coeff = Fx.diff(dof)
    final = 'v_uu[' + str(i) + '] = ' + julia_code(coeff.simplify())
    for id in range(36):
        final = final.replace( a[id], b[id] )
    for id in range(20):
        final = final.replace( c[id], d[id] )
    for id in range(15):
        final = final.replace( e[id], f[id] )
    for id in range(15):
        final = final.replace( g[id], h[id] )
    print(final)
i=0
for dof in dofsP:
    i+=1
    coeff = Fx.diff(dof)
    final = 'v_up[' + str(i) + '] = ' + julia_code(coeff.simplify())
    for id in range(36):
        final = final.replace( a[id], b[id] )
    for id in range(20):
        final = final.replace( c[id], d[id] )
    for id in range(15):
        final = final.replace( e[id], f[id] )
    for id in range(15):
        final = final.replace( g[id], h[id] )
    print(final)

v_uu[1] = (dx .^ 2 .* (b.C .* (4 * D.D11S .* b.S - 2 * D.D12S .* b.S + 3 * D.D13S .* d.S) + d.C .* (4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S)) + dy .^ 2 .* (a.C .* (4 * D.D11E .* a.E + 4 * D.D11W .* a.W - 2 * D.D12E .* a.E - 2 * D.D12W .* a.W + 3 * D.D13E .* c.E + 3 * D.D13W .* c.W) + c.C .* (4 * D.D31E .* a.E + 4 * D.D31W .* a.W - 2 * D.D32E .* a.E - 2 * D.D32W .* a.W + 3 * D.D33E .* c.E + 3 * D.D33W .* c.W))) ./ (6 * dx .^ 2 .* dy .^ 2)
v_uu[2] = (-a.C .* (4 * D.D11W .* a.W - 2 * D.D12W .* a.W + 3 * D.D13W .* c.W) - c.C .* (4 * D.D31W .* a.W - 2 * D.D32W .* a.W + 3 * D.D33W .* c.W)) ./ (6 * dx .^ 2)
v_uu[3] = (-a.C .* (4 * D.D11E .* a.E - 2 * D.D12E .* a.E + 3 * D.D13E .* c.E) - c.C .* (4 * D.D31E .* a.E - 2 * D.D32E .* a.E + 3 * D.D33E .* c.E)) ./ (6 * dx .^ 2)
v_uu[4] = (-b.C .* (4 * D.D11S .* b.S - 2 * D.D12S .* b.S + 3 * D.D13S .* d.S) - d.C .* (4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S)) ./ (6 * dy .^ 2)
v_uu[5] = 0
v_uu[6] = (-a.C .* (4 * D.D11W

In [253]:
i=0
for dof in dofsV:
    i+=1
    coeff = Fy.diff(dof)
    # print('c' + str(dof) + ' = ' + julia_code(coeff.simplify()))
    final = 'v_uu[' + str(i) + '] = ' + julia_code(coeff.simplify())
    for id in range(36):
        final = final.replace( a[id], b[id] )
    for id in range(20):
        final = final.replace( c[id], d[id] )
    for id in range(15):
        final = final.replace( e[id], f[id] )
    for id in range(15):
        final = final.replace( g[id], h[id] )
    print(final)
i=0
for dof in dofsP:
    i+=1
    coeff = Fy.diff(dof)
    final = 'v_up[' + str(i) + '] = ' + julia_code(coeff.simplify())
    for id in range(36):
        final = final.replace( a[id], b[id] )
    for id in range(20):
        final = final.replace( c[id], d[id] )
    for id in range(15):
        final = final.replace( e[id], f[id] )
    for id in range(15):
        final = final.replace( g[id], h[id] )
    print(final)

v_uu[1] = (dx .^ 2 .* (b.C .* (4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S) + d.C .* (4 * D.D21S .* b.S - 2 * D.D22S .* b.S + 3 * D.D23S .* d.S)) + dy .^ 2 .* (a.C .* (4 * D.D31E .* a.E + 4 * D.D31W .* a.W - 2 * D.D32E .* a.E - 2 * D.D32W .* a.W + 3 * D.D33E .* c.E + 3 * D.D33W .* c.W) + c.C .* (4 * D.D21E .* a.E + 4 * D.D21W .* a.W - 2 * D.D22E .* a.E - 2 * D.D22W .* a.W + 3 * D.D23E .* c.E + 3 * D.D23W .* c.W))) ./ (6 * dx .^ 2 .* dy .^ 2)
v_uu[2] = (-a.C .* (4 * D.D31W .* a.W - 2 * D.D32W .* a.W + 3 * D.D33W .* c.W) - c.C .* (4 * D.D21W .* a.W - 2 * D.D22W .* a.W + 3 * D.D23W .* c.W)) ./ (6 * dx .^ 2)
v_uu[3] = (-a.C .* (4 * D.D31E .* a.E - 2 * D.D32E .* a.E + 3 * D.D33E .* c.E) - c.C .* (4 * D.D21E .* a.E - 2 * D.D22E .* a.E + 3 * D.D23E .* c.E)) ./ (6 * dx .^ 2)
v_uu[4] = (-b.C .* (4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S) - d.C .* (4 * D.D21S .* b.S - 2 * D.D22S .* b.S + 3 * D.D23S .* d.S)) ./ (6 * dy .^ 2)
v_uu[5] = 0
v_uu[6] = (-a.C .* (4 * D.D31W

<font size= 6> Discretisation of the continuity equation </font>

In [254]:
VxE,VxW,VyN,VyS = symbols('VxE,VxW,VyN,VyS')
VyE,VyW,VxN,VxS = symbols('VyE,VyW,VxN,VxS')
PC, PC0, K = symbols('PC, PC_0, K')
aC,bC,cC,dC = symbols('aC,bC,cC,dC') 

In [255]:
dofs_pu = [VxW,VxE,VxS,VxN,VyW,VyE,VyS,VyN,PC]

In [256]:
# Velocity gradients on reference grid
eta_C = symbols('eta_C')
if free_surf==1 or free_surf==2:
    dVxdxiC  = (VxE-VxW)/dx
    dVydxiC  = (VyE-VyW)/dx
    dVxdetaC = eval_dVxdy(dVxdxiC,dVydxiC,PC,c0C,c1C,c2C)
    dVydetaC = eval_dVydy(dVxdxiC,dVydxiC,PC,d0C,d1C,d2C)
if free_surf==0:
    dVxdxiC  = (VxE-VxW)/dx
    dVydxiC  = (VyE-VyW)/dx
    dVxdetaC = (VxN-VxS)/dy
    dVydetaC = (VyN-VyS)/dy
# Velocity gradients on deformed grid
divC = d_dx2(dVxdxiC, dVxdetaC, aC,bC) + d_dy2(dVydxiC, dVydetaC,cC,dC)
Fp   = divC + (PC-PC0)/(K*dt)

In [257]:
i=0
for dof in dofs_pu:
    i+=1
    final = 'v_pu[' + str(i) + '] = ' + julia_code(Fp.diff(dof))
    for id in range(15):
        final = final.replace( e[id], f[id] )
        final = final.replace( g[id], h[id] )
    print(final)

v_pu[1] = -aC ./ dx - bC .* fs.C1C ./ dx - fs.D1C .* dC ./ dx
v_pu[2] = aC ./ dx + bC .* fs.C1C ./ dx + fs.D1C .* dC ./ dx
v_pu[3] = 0
v_pu[4] = 0
v_pu[5] = -bC .* fs.C2C ./ dx - cC ./ dx - fs.D2C .* dC ./ dx
v_pu[6] = bC .* fs.C2C ./ dx + cC ./ dx + fs.D2C .* dC ./ dx
v_pu[7] = 0
v_pu[8] = 0
v_pu[9] = bC .* fs.C0C + fs.D0C .* dC + 1 ./ (K .* dt)
